# 2025 ASAP CRN Colloborative Meeting - CRN Cloud Training Workshop Setup

## Overview
This notebook will help you set things up for the workshope.



## Steps
1. Install some nescessary tools, and define the paths in our local workspace where we will copy the files to.

2. Copy notebooks to your workspace from the resource

3. Copy the key data files you will need for the workshop to your workspace 

4. open your _workspace_ copy of the workshop notebook: [03_GOI_analysis.ipynb](~/workspace/workshop_files/03_GOI_analysis.ipynb)


________________


In [1]:
# Use pathlib for file path manipulation
from pathlib import Path 


# matplotlib and seaborn a pythonic alternative to plotnine
try:
    import seaborn as sns
    import matplotlib.pyplot as plt
except ImportError as e:
    print("Error -> ", e)
    print("Installing seaborn or matplotlib")
    !pip install matplotlib seaborn
    import seaborn as sns
    import matplotlib.pyplot as plt


    
#  plotnine: a ggplot2-compatible Python plotting package
try:
    import plotnine as p9
except ImportError as e:
    print("Error -> ", e)
    print("Installing plotnine")
    !pip install plotnine
    import plotnine as p9

try:
    import scanpy as sc
except ImportError as e:
    print("Error -> ", e)
    print("Installing scanpy")
    !pip install scanpy
    import scanpy as sc


try:
    from scipy.stats import norm
    import statsmodels.formula.api as smf    
    from statsmodels.stats.multitest import multipletests

except ImportError as e:
    print("Error -> ", e)
    print("Installing scipy and statsmodel")
    !pip install scanpy statsmodel
    from scipy.stats import norm
    import statsmodels.formula.api as smf    
    from statsmodels.stats.multitest import multipletests

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime



time: 174 μs (started: 2025-03-06 08:10:23 +00:00)


## Imports

If we can't import the python modules we will need we can go ahead and install them now.

###  ASAP CRN data paths
First, let's build the paths to our data. 

In [2]:
# Workspace Path
HOME_PATH = Path.home()
WS_PATH =  HOME_PATH / "workspace"
if not WS_PATH.exists():
    print(f"{WS_PATH} doesn't exist. We need to remount our resources")
    !wb resource mount    

time: 673 μs (started: 2025-03-06 08:10:23 +00:00)


#### Workshop/Notebook Artifact Path

This is the path to where the artifacts we've created in these notebooks can be found in case you want to skip ahead.  Particularly for the workshop.  In which case you can replace the `local_data_path` with `WORKSHOP_PATH`.

In [3]:
WORKSHOP_PATH = WS_PATH / "release_resources/asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025" 

!ls {WORKSHOP_PATH}


data  notebooks
time: 562 ms (started: 2025-03-06 08:10:23 +00:00)


#### Local Data Path
Lets also define a path for copying our data files and exporting intermediate analysis artifacts to your workspace.  In this example we'll make a "workshop_files" which are persistent in our Verily Workbench Workspace. 

In [4]:
local_data_path = WS_PATH / "workshop_files"

if not local_data_path.exists():
    local_data_path.mkdir(parents=True)


local_nb_path = local_data_path / "notebooks"
if not local_nb_path.exists():
    local_nb_path.mkdir(parents=True)


time: 1.48 ms (started: 2025-03-06 08:10:38 +00:00)


## copy workshop files to your local workspace

Now that we've defined our local_data_path we can start copying. First lets get our notebooks. 


In [5]:
!cp  {WORKSHOP_PATH/"notebooks/*"} {local_nb_path}

cp: cannot create regular file '/home/jupyter/workspace/workshop_files/notebooks/workspace_example1.png': Permission denied
time: 201 ms (started: 2025-03-06 08:10:58 +00:00)


In [10]:
# fix writeability
!chmod u+w {local_nb_path/"*"} ; chmod u+w {local_data_path}

time: 124 ms (started: 2025-03-06 08:44:04 +00:00)


In [12]:
DATASETS_PATH = WS_PATH / "01_PMDBS_scRNAseq_Datasets"
workflow = "pmdbs_sc_rnaseq"
dataset_team = "cohort"
dataset_source = "pmdbs"
dataset_type = "sc-rnaseq"
bucket_name = f"asap-curated-{dataset_team}-{dataset_source}-{dataset_type}"
dataset_name = f"asap-{dataset_team}-{dataset_source}-{dataset_type}"
dataset_path = DATASETS_PATH / bucket_name / workflow
cohort_analysis_path = dataset_path / "cohort_analysis"

adata_filename = cohort_analysis_path / f"asap-{dataset_team}.final_adata.h5ad"
local_adata_filename = local_data_path / f"asap-{dataset_team}.final_adata.h5ad"

!cp {adata_filename} {local_adata_filename}
    

time: 2min 44s (started: 2025-03-06 08:56:51 +00:00)


Now the data files which are made in parts 1 and 2:   [01_basic_EDA.ipynb](./01_basic_EDA.ipynb) ,  [02_dataset_composition.ipynb](./02_dataset_composition.ipynb)

In [11]:
!gsutil -m cp \
  "gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/GP2 European GWAS Manuscript Tables - Table 4.csv" \
  "gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/asap-cohort.final_metadata.csv" \
  "gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/asap-cohort.full_frontal_ctx_case_control_samples.h5ad" \
{local_data_path}





Copying gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/GP2 European GWAS Manuscript Tables - Table 4.csv...
Copying gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/asap-cohort.final_metadata.csv...
Copying gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/asap-cohort.full_cortical_ctx_case_control_samples.h5ad...
\ [3/3 files][966.8 MiB/966.8 MiB] 100% Done  82.8 MiB/s ETA 00:00:00           
Operation completed over 3 objects/966.8 MiB.                                    
time: 24.3 s (started: 2025-03-06 08:44:54 +00:00)


Finally some psuedobulk files for part 4: [04_DE_analysis.ipynb](./04_DE_analysis.ipynb)



In [ ]:
!gsutil -m cp \
  "gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/asap-cohort.full_frontal_ctx_psuedobulk.h5ad" \
  "gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/asap-cohort.full_frontal_ctx_psuedobulk_de.csv" \
  "gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/meta_analysis_results_2025-02-24.csv" \
  "gs://asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025/data/2025-02-24.asap-cohort.full_frontal_ctx_psuedobulk.h5ad" \
  .

In [ ]:
## workshop data
# 'workshop/asap-cohort.frontal_ctx_case_control_samples.h5ad'
# 'workshop/asap-cohort.full_frontal_ctx_case_control_samples.h5ad'
# 'workshop/asap-cohort.asap-cohort-dataset-metadata.csv
# "workshop/merged_adata_object.h5ad"

# # make sure we have all the files we need:
# - 'workspace/01_PMDBS_scRNAseq_Datasets/asap-curated-cohort-pmdbs-sc-rnaseq/pmdbs_sc_rnaseq/cohort_analysis/asap-cohort.final_metadata.csv'
# - 'workspace/01_PMDBS_scRNAseq_Datasets/asap-curated-cohort-pmdbs-sc-rnaseq/pmdbs_sc_rnaseq/cohort_analysis/asap-cohort.final_adata.h5ad'
# - 'workspace/01_PMDBS_scRNAseq_Datasets/asap-curated-cohort-pmdbs-sc-rnaseq/pmdbs_sc_rnaseq/cohort_analysis/asap-cohort.merged_adata_object.h5ad'

# psuedobulk data

cp: cannot stat '/home/ergonyc/workspace/release_resources/asap-crn-cloud-release-resources/release-artifacts/2025_CRN_CM_Workshop_Resources_03122025': No such file or directory
time: 187 ms (started: 2025-03-05 21:01:03 -08:00)
